# Next Word Prediction:

In [1]:
# Check if GPU is available
import tensorflow as tf
if tf.config.list_physical_devices('GPU'):
    print('GPU is available')
    device = '/GPU:0'
else:
    print('GPU is not available')
    device = '/CPU:0'

GPU is available


In [2]:
! pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 KB 4.4 MB/s eta 0:00:00


In [3]:
# %%
import torch
import string

In [4]:
from transformers import AutoTokenizer

In [5]:
from transformers import AutoModelForMaskedLM
bert_tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
bert_model = AutoModelForMaskedLM.from_pretrained("bert-base-uncased").eval()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
from transformers import AutoModelForCausalLM
xlnet_tokenizer = AutoTokenizer.from_pretrained("xlnet-base-cased")
xlnet_model = AutoModelForCausalLM.from_pretrained("xlnet-base-cased").eval()

In [7]:
from transformers import AutoModelForMaskedLM
xlmroberta_tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")
xlmroberta_model = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base").eval()

In [8]:
from transformers import AutoModel
bart_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
bart_model = AutoModel.from_pretrained("facebook/bart-large").eval()

In [9]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
electra_tokenizer = AutoTokenizer.from_pretrained("google/electra-base-generator")
electra_model = AutoModelForMaskedLM.from_pretrained("google/electra-base-generator").eval()

Some weights of ElectraForMaskedLM were not initialized from the model checkpoint at google/electra-base-generator and are newly initialized: ['electra.embeddings.position_ids']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
from transformers import RobertaTokenizer, RobertaForMaskedLM
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForMaskedLM.from_pretrained('roberta-base').eval()

In [11]:
top_k = 10


def decode(tokenizer, pred_idx, top_clean):
    ignore_tokens = string.punctuation + '[PAD]'
    tokens = []
    for w in pred_idx:
        token = ''.join(tokenizer.decode(w).split())
        if token not in ignore_tokens:
            tokens.append(token.replace('##', ''))
    return '\n'.join(tokens[:top_clean])

In [12]:
def encode(tokenizer, text_sentence, add_special_tokens=True):
    text_sentence = text_sentence.replace('<mask>', tokenizer.mask_token)
    # if <mask> is the last token, append a "." so that models dont predict punctuation.
    if tokenizer.mask_token == text_sentence.split()[-1]:
        text_sentence += ' .'

    input_ids = torch.tensor([tokenizer.encode(text_sentence, add_special_tokens=add_special_tokens)])
    mask_idx = torch.where(input_ids == tokenizer.mask_token_id)[1].tolist()[0]
    return input_ids, mask_idx

In [13]:
def get_all_predictions(text_sentence, top_clean=5):
    # ========================= BERT =================================
    print(text_sentence)
    input_ids, mask_idx = encode(bert_tokenizer, text_sentence)
    with torch.no_grad():
        predict = bert_model(input_ids)[0]
    bert = decode(bert_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLNET =================================
    input_ids, mask_idx = encode(xlnet_tokenizer, text_sentence, False)
    perm_mask = torch.zeros((1, input_ids.shape[1], input_ids.shape[1]), dtype=torch.float)
    perm_mask[:, :, mask_idx] = 1.0  # Previous tokens don't see last token
    target_mapping = torch.zeros((1, 1, input_ids.shape[1]), dtype=torch.float)  # Shape [1, 1, seq_length] => let's predict one token
    target_mapping[0, 0, mask_idx] = 1.0  # Our first (and only) prediction will be the last token of the sequence (the masked token)

    with torch.no_grad():
        predict = xlnet_model(input_ids, perm_mask=perm_mask, target_mapping=target_mapping)[0]
    xlnet = decode(xlnet_tokenizer, predict[0, 0, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= XLM ROBERTA BASE =================================
    input_ids, mask_idx = encode(xlmroberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = xlmroberta_model(input_ids)[0]
    xlm = decode(xlmroberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= BART =================================
    input_ids, mask_idx = encode(bart_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = bart_model(input_ids)[0]
    bart = decode(bart_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ELECTRA =================================
    input_ids, mask_idx = encode(electra_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = electra_model(input_ids)[0]
    electra = decode(electra_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    # ========================= ROBERTA =================================
    input_ids, mask_idx = encode(roberta_tokenizer, text_sentence, add_special_tokens=True)
    with torch.no_grad():
        predict = roberta_model(input_ids)[0]
    roberta = decode(roberta_tokenizer, predict[0, mask_idx, :].topk(top_k).indices.tolist(), top_clean)

    return {'bert': bert,
            'xlnet': xlnet,
            'xlm': xlm,
            'bart': bart,
            'electra': electra,
            'roberta': roberta}

In [14]:
import pandas as pd

# with mask in the middle
sentence = "I went to the <mask> to buy some milk"
print("The Top 5 predictions for the sentence: ")

models = ['bert', 'xlnet', 'xlm', 'bart', 'electra', 'roberta']
df = pd.DataFrame(columns=models)
predictions = get_all_predictions(sentence, top_clean=5)
for model in models:
    df.loc[0, model] = predictions[model].split('\n')[0]
    df.loc[1, model] = predictions[model].split('\n')[1]
    df.loc[2, model] = predictions[model].split('\n')[2]
    df.loc[3, model] = predictions[model].split('\n')[3]
    df.loc[4, model] = predictions[model].split('\n')[4]

df

The Top 5 predictions for the sentence: 
I went to the <mask> to buy some milk


,bert,xlnet,xlm,bart,electra,roberta
0,store,to,supermarket,One,store,store
1,supermarket,the,store,The,supermarket,supermarket
2,market,market,market,big,pharmacy,market
3,shop,store,farm,County,market,chemist
4,bookstore,one,shop,</s>,hospital,pharmacy


In [15]:
# with mask at the end
sentence = "I went to the <mask>"
print("The Top 5 predictions for the sentence: ", sentence)

models = ['bert', 'xlnet', 'xlm', 'bart', 'electra', 'roberta']
df = pd.DataFrame(columns=models)
predictions = get_all_predictions(sentence, top_clean=5)
for model in models:
    df.loc[0, model] = predictions[model].split('\n')[0]
    df.loc[1, model] = predictions[model].split('\n')[1]
    df.loc[2, model] = predictions[model].split('\n')[2]
    df.loc[3, model] = predictions[model].split('\n')[3]
    df.loc[4, model] = predictions[model].split('\n')[4]

df

The Top 5 predictions for the sentence:  I went to the <mask>
I went to the <mask>


,bert,xlnet,xlm,bart,electra,roberta
0,bathroom,the,bar,One,bathroom,hospital
1,door,a,bathroom,The,hospital,library
2,kitchen,to,supermarket,big,hotel,airport
3,window,following,bank,</s>,pharmacy,mall
4,closet,last,kitchen,its,doctor,park


In [16]:
# without mask
sentence = "I went to the store to buy some milk"
sentence = sentence + " <mask>"
print("The Top 5 predictions for the sentence: ", sentence)

models = ['bert', 'xlnet', 'xlm', 'bart', 'electra', 'roberta']
df = pd.DataFrame(columns=models)
predictions = get_all_predictions(sentence, top_clean=5)
for model in models:
    df.loc[0, model] = predictions[model].split('\n')[0]
    df.loc[1, model] = predictions[model].split('\n')[1]
    df.loc[2, model] = predictions[model].split('\n')[2]
    df.loc[3, model] = predictions[model].split('\n')[3]
    df.loc[4, model] = predictions[model].split('\n')[4]
df

The Top 5 predictions for the sentence:  I went to the store to buy some milk <mask>
I went to the store to buy some milk <mask>


,bert,xlnet,xlm,bart,electra,roberta
0,shakes,milk,myself,One,again,today
1,instead,stock,cream,The,products,for
2,too,and,powder,2,cream,yesterday
3,again,product,today,action,instead,powder
4,myself,a,...,between,too,and
